In [ ]:
import pandas as pd

In [ ]:
train_csv = pd.read_csv('train.csv', delimiter=';', names=["id", "user_id", "movie_id", "rating"])

In [ ]:
train_df = train_csv.pivot(index='movie_id',columns='user_id',values='rating')
train_df

In [ ]:
correlated_users = train_df.corr(method ='pearson')
correlated_users

In [ ]:
task_csv = pd.read_csv('task.csv', delimiter=';', names=["id", "user_id", "movie_id", "rating"])

In [ ]:
similar_users_temp = correlated_users[673]
# similar_users_temp = correlated_users[1642]

similar_users_temp = similar_users_temp.sort_values(ascending=False)[1:]
similar_users_temp.head()

user_id
12   NaN
19   NaN
24   NaN
31   NaN
52   NaN
Name: 673, dtype: float64

In [ ]:
def get_avg(values):
  m,l = 0, 0
  for val in values:
    l += val[0]*val[1]
    m += val[0]
  return str(int(round(l/m)))

In [ ]:
def predict_rating(movie_id, similar_users, user_id):
  movie_raitings = train_csv.loc[train_csv['movie_id'] == movie_id]
  similar_users_df = pd.DataFrame({'user_id':similar_users.index, 'similarity':similar_users.values})
  similar_users_df = pd.merge(similar_users_df, movie_raitings, on="user_id")
  winners = similar_users_df.loc[pd.notnull(similar_users_df.similarity)].head(n=3)

  if len(winners) > 0:
    return get_avg(winners[['similarity', 'rating']].values.tolist())
  else:
    user_ratings = train_csv.loc[train_csv['user_id'] == user_id]
    return str(int(round(user_ratings['rating'].mean())))

In [ ]:
predict_rating(68, similar_users_temp, 673)

'3'

In [ ]:
temp_user_id = -1
for index, row in task_csv.iterrows():
  if temp_user_id != int(row['user_id']):
    similar_users = correlated_users[int(row['user_id'])]
    similar_users = similar_users.sort_values(ascending=False)[1:]
  task_csv.at[index, 'rating'] = predict_rating(int(row['movie_id']), similar_users, int(row['user_id']))
  temp_user_id = int(row['user_id'])
  print(index, row['user_id'])
task_csv

In [ ]:
task_csv.to_csv('submission.csv', sep=';', encoding='utf-8', index=False, header=False)

Dobra - sprawdzamy drugi sposób

In [ ]:
import pandas as pd

In [ ]:
task_csv = pd.read_csv('task.csv', delimiter=';', names=["id", "user_id", "movie_id", "rating"])
train_csv = pd.read_csv('train.csv', delimiter=';', names=["id", "user_id", "movie_id", "rating"])
train_df = train_csv.pivot(index='movie_id',columns='user_id',values='rating')
train_df

In [ ]:
def get_similar_users(user_id):
  users = []
  for usr_id, ratings in train_df.iteritems():
    if usr_id != user_id:
      users.append((usr_id, sum(abs(train_df[user_id] - ratings).fillna(0))))
  
  return sorted(users, key=lambda x: x[1])
temp_users = get_similar_users(1642)

In [ ]:
def average(lst):
    return sum(lst) / len(lst)

In [ ]:
def predict_user_rating(movie_id, similar_users, user_id):
  best_ratings = []
  for similar_user in similar_users:
    movie_rating = train_csv[(train_csv['movie_id'] == movie_id) & (train_csv['user_id'] == similar_user[0])]['rating']
    if len(movie_rating.values) > 0:
      best_ratings.append(movie_rating.values[0])
    if len(best_ratings) >= 3:
      return str(int(round(average(best_ratings))))
  return str(int(round(average(best_ratings))))

predict_user_rating(137, temp_users, 1642)

'4'

In [ ]:
temp_user_id = -1
for index, row in task_csv.iterrows():
  if temp_user_id != int(row['user_id']):
    similar_users = get_similar_users(int(row['user_id']))
  task_csv.at[index, 'rating'] = predict_user_rating(int(row['movie_id']), similar_users, int(row['user_id']))
  temp_user_id = int(row['user_id'])
  print(index, row['user_id'])
task_csv

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
5740 708.0
5741 708.0
5742 708.0
5743 708.0
5744 708.0
5745 708.0
5746 708.0
5747 708.0
5748 708.0
5749 708.0
5750 708.0
5751 708.0
5752 708.0
5753 708.0
5754 708.0
5755 708.0
5756 708.0
5757 708.0
5758 708.0
5759 708.0
5760 1196.0
5761 1196.0
5762 1196.0
5763 1196.0
5764 1196.0
5765 1196.0
5766 1196.0
5767 1196.0
5768 1196.0
5769 1196.0
5770 1196.0
5771 1196.0
5772 1196.0
5773 1196.0
5774 1196.0
5775 1196.0
5776 1196.0
5777 1196.0
5778 1196.0
5779 1196.0
5780 1196.0
5781 1196.0
5782 1196.0
5783 1196.0
5784 1196.0
5785 1196.0
5786 1196.0
5787 1196.0
5788 1196.0
5789 1196.0
5790 254.0
5791 254.0
5792 254.0
5793 254.0
5794 254.0
5795 254.0
5796 254.0
5797 254.0
5798 254.0
5799 254.0
5800 254.0
5801 254.0
5802 254.0
5803 254.0
5804 254.0
5805 254.0
5806 254.0
5807 254.0
5808 254.0
5809 254.0
5810 254.0
5811 254.0
5812 254.0
5813 254.0
5814 254.0
5815 254.0
5816 254.0
5817 254.0
5818 254.0
5819 254.0
5820 1686.0
5821 1686.0
5

,id,user_id,movie_id,rating
0,32221,1642,68,4.0
1,32222,1642,69,3.0
2,32223,1642,142,3.0
3,32224,1642,77,3.0
4,32225,1642,128,4.0
...,...,...,...,...
10735,42956,1455,51,4.0
10736,42957,1455,117,3.0
10737,42958,1455,191,3.0
10738,42959,1455,125,3.0


In [ ]:
task_csv.to_csv('submission.csv', sep=';', encoding='utf-8', index=False, header=False)